In [16]:
print('\n\n\n=======================\nALGORITHMIC TRADING BOT\
    \nTEST ENVIRONMENT\n=======================\n\n\n')

balance = 10000
print(f'Starting Balance: ${balance}')

print('\nAlgorithm Rules:\nTrade Size = 5% of balance.\nTP/SL = 1 ATR\nDaily Profit Target = 2% Profit')




ALGORITHMIC TRADING BOT    
TEST ENVIRONMENT



Starting Balance: $10000

Algorithm Rules:
Trade Size = 5% of balance.
TP/SL = 1 ATR
Daily Profit Target = 2% Profit
